# Competition Objective is to detect fraud in transactions; 

## Data


In this competition you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target ```isFraud```.

The data is broken into two files **identity** and **transaction**, which are joined by ```TransactionID```. 

> Note: Not all transactions have corresponding identity information.

**Transaction variables**

- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

**Categorical Features - Transaction**

- ProductCD
- emaildomain
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

**Categorical Features - Identity**

- DeviceType
- DeviceInfo
- id_12 - id_38

**The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).**


# 1. Importation and memory reduction
## 1.1. Importing necessary libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import datetime


# Standard plotly imports
#import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets
import plotly.express as px
import plotly.tools as tls
from plotly.subplots import make_subplots

from plotly.offline import iplot, init_notebook_mode
#import cufflinks
#import cufflinks as cf
import plotly.figure_factory as ff


# Using plotly + cufflinks in offline mode
init_notebook_mode(connected=True)
#cufflinks.go_offline(connected=True)

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold, GridSearchCV
from xgboost import XGBClassifier
import xgboost as xgb
import catboost as cb

## Hyperopt modules
from functools import partial

import os
import gc
import time

In [ ]:
!jupyter nbextension enable --py widgetsnbextension


## 1.2. Importing train datasets

In [ ]:
gc.collect()
df_train = pd.read_pickle("/kaggle/input/fraud-detection/df_train_v2.pkl")
df_test = pd.read_pickle("/kaggle/input/fraud-detection/df_test_v2.pkl")
sample_submission = pd.read_csv('/kaggle/input/fraud-detection/sample_submission.csv')

In [ ]:
print(df_train.shape)
print(df_test.shape)

## 1.3. Memory reduction

In [ ]:
def resumetable(df):
    n = df.shape[0]
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values  
    summary['Missing %'] = round(summary['Missing'] / n * 100,2)
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values


    return summary


To see the output of the Resume Table, click to see the output 

# 2. Preprocessing

In [ ]:
df_train.head()

In [ ]:
categorical_features = []
for col in df_train.columns.drop('isFraud') :
    if (df_train[col].dtype == 'object' 
        or df_test[col].dtype=='object' 
        or df_train[col].dtype == 'bool') :
        categorical_features.append(col)
categorical_features.extend(['_merge', '_Weekdays',	'_Hours',	'_Days'])

In [ ]:
categorical_resume = resumetable(df_train[categorical_features])
categorical_resume

In [ ]:
# Label Encoding
categorical_features_v2 = categorical_features.copy()
for f in categorical_features:
    if float(categorical_resume.loc[categorical_resume['Name']==f, 'Uniques']) > 8 :
        categorical_features_v2.remove(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(df_train[f].values) + list(df_test[f].values))
        df_train[f] = lbl.transform(list(df_train[f].values))
        df_test[f] = lbl.transform(list(df_test[f].values))  

In [ ]:
categorical_resume_v2 = resumetable(df_train[categorical_features_v2])
categorical_resume_v2

In [ ]:
Y_train = df_train['isFraud']
df_train.drop(['isFraud','TransactionID'], axis = 1, inplace = True)
df_test.drop('TransactionID', axis = 1, inplace = True)

In [ ]:
categorical_index = [df_train.columns.get_loc(key) for key in categorical_features_v2]


# 3. Modeling

In [ ]:
train = cb.Pool(df_train, Y_train, cat_features=categorical_index)
test = cb.Pool(df_test, cat_features=categorical_index)
del df_train, Y_train, df_test
gc.collect()


In [ ]:
params_cv = {'depth': 12,
             'learning_rate' : 0.03,
             'l2_leaf_reg': 3,
             'loss_function': "Logloss", 
             'eval_metric' : 'AUC',
             'iterations': 500}
cb.cv(train,
      params_cv,
      fold_count=3,
      stratified=True,
      plot="True")